In [1]:
import gym
import random
import tflearn
from statistics import median, mean
from collections import Counter
import tensorflow as tf
import numpy as np
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
env_name = "CartPole-v1"
env = gym.make(env_name)
env.reset()
req_score = 50
LR=  1e-3
print("obs space: " ,env.observation_space)
print("act space: " ,env.action_space)

obs space:  Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
act space:  Discrete(2)


In [3]:
def population(): 
    training_data = []
    scores = []
    accepted_score = []
    
    for _ in range(10000):
        score = 0 
        memmory = []
        prev_obs = []
        
        for _ in range(500): 
            action = random.randrange(0,2)
            obs,reward,done,info = env.step(action)
            
            if len(prev_obs) > 0: 
                memmory.append([prev_obs, action])
            prev_obs = obs 
            score += reward
            if done: break
                    
        if score > req_score: 
            accepted_score.append(score)
            for d in memmory:
                if d[1] ==0: 
                    output = [0,1]
                elif d[1]==1: 
                    output = [1,0]
                    
                training_data.append([d[0],output])
        env.reset()
        scores.append(score)
        
    print('Average accepted score:',mean(accepted_score))
    print('Median score for accepted scores:',median(accepted_score))
    print(Counter(accepted_score))
        
    return training_data

In [4]:
population()

Average accepted score: 62.62950819672131
Median score for accepted scores: 60.0
Counter({52.0: 28, 57.0: 20, 51.0: 18, 54.0: 18, 56.0: 17, 59.0: 15, 53.0: 14, 61.0: 13, 62.0: 13, 71.0: 12, 55.0: 12, 60.0: 11, 63.0: 11, 58.0: 10, 64.0: 8, 65.0: 7, 69.0: 7, 73.0: 6, 70.0: 5, 68.0: 5, 79.0: 5, 72.0: 4, 67.0: 4, 66.0: 4, 82.0: 3, 81.0: 3, 78.0: 3, 75.0: 3, 74.0: 3, 84.0: 3, 77.0: 3, 97.0: 2, 96.0: 2, 85.0: 2, 76.0: 2, 94.0: 2, 87.0: 1, 91.0: 1, 114.0: 1, 99.0: 1, 83.0: 1, 88.0: 1, 89.0: 1})


[[array([-0.00652335,  0.21085601,  0.02600296, -0.24398933]), [0, 1]],
 [array([-0.00230623,  0.01537249,  0.02112318,  0.05678084]), [1, 0]],
 [array([-0.00199878,  0.2101853 ,  0.02225879, -0.22916343]), [0, 1]],
 [array([0.00220493, 0.01475245, 0.01767552, 0.07045676]), [1, 0]],
 [array([ 0.00249998,  0.2096166 ,  0.01908466, -0.21659746]), [0, 1]],
 [array([0.00669231, 0.01422709, 0.01475271, 0.08204398]), [0, 1]],
 [array([ 0.00697685, -0.1811032 ,  0.01639359,  0.37934466]), [1, 0]],
 [array([0.00335479, 0.01378216, 0.02398048, 0.09187541]), [1, 0]],
 [array([ 0.00363043,  0.20855233,  0.02581799, -0.19314626]), [1, 0]],
 [array([ 0.00780148,  0.40329562,  0.02195507, -0.4775742 ]), [1, 0]],
 [array([ 0.01586739,  0.59810082,  0.01240358, -0.7632573 ]), [0, 1]],
 [array([ 0.0278294 ,  0.40281025, -0.00286156, -0.46669742]), [0, 1]],
 [array([ 0.03588561,  0.20772884, -0.01219551, -0.17491781]), [0, 1]],
 [array([ 0.04004019,  0.01278353, -0.01569387,  0.11389301]), [0, 1]],
 [ar

In [5]:
def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model


def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

training_data = population()

model = train_model(training_data)



Training Step: 1549  | total loss: 0.66683 | time: 1.349s
| Adam | epoch: 005 | loss: 0.66683 - acc: 0.5906 -- iter: 19776/19795
Training Step: 1550  | total loss: 0.66458 | time: 1.354s
| Adam | epoch: 005 | loss: 0.66458 - acc: 0.5941 -- iter: 19795/19795
--
